# Sales Report using SQL + Pandas 📊

In [1]:
import sys
!{sys.executable} -m pip install fpdf -q
!{sys.executable} -m pip install pandas -q
!{sys.executable} -m pip install plotly-express -q
!{sys.executable} -m pip install kaleido -q

'C:\Users\Kyaw' is not recognized as an internal or external command,
operable program or batch file.
'C:\Users\Kyaw' is not recognized as an internal or external command,
operable program or batch file.
'C:\Users\Kyaw' is not recognized as an internal or external command,
operable program or batch file.
'C:\Users\Kyaw' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd  
import plotly.express as px  
from fpdf import FPDF  

In [3]:
plotly_template = "presentation"

In [4]:
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "sales.db"
output_dir = current_dir / "output"

output_dir.mkdir(parents=True, exist_ok=True)

## Total Sales by Month

In [5]:
conn = sqlite3.connect(database_path)

In [6]:
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC
'''
df = pd.read_sql_query(query, conn)

In [7]:
print(df)

      sale_date  total_sales
0    2022-01-02          150
1    2022-01-03         1630
2    2022-01-04          350
3    2022-01-05          225
4    2022-01-06          700
..          ...          ...
341  2022-12-27          520
342  2022-12-28         1095
343  2022-12-29         1450
344  2022-12-30          880
345  2022-12-31          225

[346 rows x 2 columns]


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    346 non-null    object
 1   total_sales  346 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.5+ KB


In [9]:
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    346 non-null    datetime64[ns]
 1   total_sales  346 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.5 KB


In [10]:
df = df.set_index('sale_date')
df.head(3)

,total_sales
sale_date,
2022-01-02,150
2022-01-03,1630
2022-01-04,350


In [11]:
df_monthly = df.resample('M').sum()
df_monthly

C:\Users\Kyaw Lin\AppData\Local\Temp\ipykernel_22496\2617197382.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = df.resample('M').sum()


,total_sales
sale_date,
2022-01-31,20555
2022-02-28,20900
2022-03-31,17895
2022-04-30,22205
2022-05-31,21000
2022-06-30,25015
2022-07-31,24970
2022-08-31,25265
2022-09-30,24445


In [12]:
df_monthly['month_name'] = df_monthly.index.strftime('%b')
df_monthly

,total_sales,month_name
sale_date,,
2022-01-31,20555,Jan
2022-02-28,20900,Feb
2022-03-31,17895,Mar
2022-04-30,22205,Apr
2022-05-31,21000,May
2022-06-30,25015,Jun
2022-07-31,24970,Jul
2022-08-31,25265,Aug
2022-09-30,24445,Sep


In [13]:
fig = px.bar(df_monthly,
             x='month_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

fig.update_layout(
    title='Total Sales by Month',
    xaxis_title='Month',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

fig.show()

fig.write_image(output_dir / 'monthly_sales.png',
                width=1200,
                height=400,
                scale=4)

## Total Sales by Product

In [14]:
query = '''
SELECT p.product_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name
'''
df = pd.read_sql_query(query, conn)
df

,product_name,total_sales
0,Product A,57350
1,Product B,28625
2,Product C,76800
3,Product D,43240
4,Product E,67800


In [17]:
fig = px.bar(df,
             x='product_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')

fig.update_layout(
    title='Total Sales by Product',
    xaxis_title='Product',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

fig.show()

fig.write_image(output_dir / 'product_sales.png',
                width=1200,
                height=400,
                scale=4)

## Top Customer by Sales

In [18]:
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)
df

,customer_name,total_sales
0,Melissa Smith,2575
1,Ryan Holmes,1775
2,Michael Smith,1735
3,Amanda Moore,1515
4,Shane Hansen,1485
5,Laurie Adams,1465
6,Debra Jones,1450
7,Bryan Johnston,1450
8,Regina Williams,1425
9,John Baker,1355


In [26]:
df['first_name'] = df['customer_name'].str.split().str[0]

fig = px.bar(df,
             x='first_name',
             y='total_sales',
             template=plotly_template,
             text='total_sales')


fig.update_layout(
    title='Top Customers by Sales',
    xaxis_title='Customer',
    yaxis_title='Total Sales ($)',
    yaxis_tickprefix='$',
)

fig.show()

fig.write_image(output_dir / 'customer_sales.png',
                width=1200,
                height=400,
                scale=4)

## Create PDF Report

In [30]:
font_color = (64, 64, 64)

chart_filenames = [str(chart_path) for chart_path in output_dir.glob("*.png")]

pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 24)

title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(0, 20, title, align='C', ln=1)

for chart_filename in chart_filenames:
    pdf.ln(10)  
    pdf.image(chart_filename, x=None, y=None, w=pdf.w - 20, h=0)

pdf.output(output_dir / "sales_report.pdf", "F")

''

### Execute the query and load results into a Pandas DataFrame

In [27]:
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales,
  CASE 
    WHEN SUM(sales.total_price) > 1000 THEN 'High Value'
    WHEN SUM(sales.total_price) > 500 THEN 'Medium Value'
    ELSE 'Low Value'
  END as customer_segment
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
ORDER BY total_sales DESC
'''

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales,customer_segment
0,178,Ryan Holmes,1775,High Value
1,788,Melissa Smith,1620,High Value
2,250,Amanda Moore,1515,High Value
3,955,Shane Hansen,1485,High Value
4,251,Laurie Adams,1465,High Value
...,...,...,...,...
621,990,Kathy Miller,25,Low Value
622,541,Gloria Browning,25,Low Value
623,475,Robin Burns,25,Low Value
624,173,Drew Orr,25,Low Value


### Execute the query and load results into a Pandas DataFrame

In [28]:
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
'''

df = pd.read_sql_query(query, conn)
df

,customer_id,customer_name,total_sales
0,2,Cindy Espinoza,855
1,4,Jean Garcia,120
2,5,Jimmy Phillips,360
3,6,James Deleon,155
4,13,Rebecca Carter,120
...,...,...,...
621,994,Jennifer Allen,225
622,995,Kyle Anderson,150
623,996,Stephanie Wolf,150
624,998,Eddie Lee,500


In [29]:
bins = [0, 500, 1000, float('inf')]
labels = ['Low Value', 'Medium Value', 'High Value']
df['customer_segment'] = pd.cut(df['total_sales'], bins=bins, labels=labels)

df = df.sort_values(by='total_sales', ascending=False)
df

,customer_id,customer_name,total_sales,customer_segment
112,178,Ryan Holmes,1775,High Value
492,788,Melissa Smith,1620,High Value
167,250,Amanda Moore,1515,High Value
598,955,Shane Hansen,1485,High Value
168,251,Laurie Adams,1465,High Value
...,...,...,...,...
300,475,Robin Burns,25,Low Value
618,990,Kathy Miller,25,Low Value
59,102,Amanda Lawrence,25,Low Value
107,173,Drew Orr,25,Low Value


In [38]:
conn.close()